In [10]:
import os
import copy
import numpy as np
import pandas as pd
import time
import datetime
import math
import matplotlib.pyplot as plt
from uniswappy import *
import plotly.graph_objects as go

In [11]:
n_rows = 10_000

In [12]:

high = pd.read_csv('weth_dai_events_3000.csv')[:n_rows]

high['dollar_price'] = 1 / ((high.sqrtPriceX96.astype(float)) / 2**96) ** 2
high['dollar_price'] = high.dollar_price.shift(-1).ffill()
high['lp_tuple'] = list(zip(high.liquidity_provider, high.tickLower, high.tickUpper))

low = pd.read_csv('weth_dai_events_500.csv')[:n_rows]
low['dollar_price'] = 1 / ((low.sqrtPriceX96.astype(float)) / 2**96) ** 2
low['dollar_price'] = low.dollar_price.shift(-1).ffill()
low['tp_tuple'] = list(zip(low.liquidity_provider, low.tickLower, low.tickUpper))
low.evt_block_time = pd.to_datetime(low.evt_block_time)
high.evt_block_time = pd.to_datetime(high.evt_block_time)

max_time = min(low.evt_block_time.max(), high.evt_block_time.max())
min_time = max(low.evt_block_time.min(), high.evt_block_time.min())




In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=high['evt_block_time'], y=high['dollar_price'], mode='lines', name='High Fee Pool (0.3%)'))
fig.add_trace(go.Scatter(x=low['evt_block_time'], y=low['dollar_price'], mode='lines', name='Low Fee Pool (0.05%)'))
fig.update_layout(title="WETH/DAI Dollar Prices", xaxis_title="Block Time", yaxis_title="Dollar Price", template="plotly_white")
fig.show()

First pool: 

WETH / DAI fee tier = 3000

0xC2e9F25Be6257c210d7Adf0D4Cd6E3E881ba25f8

In [14]:
fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

dai = ERC20("DAI", "0x111")
weth = ERC20("WETH", "0x09")

exchg_data = UniswapExchangeData(tkn0 = weth, tkn1 = dai, symbol="LP",
                                   address="0x011", version = 'V3',
                                   tick_spacing = tick_spacing,
                                   fee = fee)

factory = UniswapFactory("SYS pool factory", "0x2")
lp_medium_fee = factory.deploy(exchg_data)

lp_medium_fee.precision = 'GWEI'
user_nm = 'user0'

init_sqrt_price_96 = int(1354707084081889255546596745)

lp_medium_fee.initialize(init_sqrt_price_96)
print(lp_medium_fee.get_price(dai))

None


Second Pool: 0x60594a405d53811d3BC4766596EFD80fd545A270

WETH / DAI

Fee tier = 500

In [15]:
fee = UniV3Utils.FeeAmount.LOW
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

dai = ERC20("DAI", "0x111")
weth = ERC20("WETH", "0x09")

exchg_data = UniswapExchangeData(tkn0 = weth, tkn1 = dai, symbol="LP",
                                   address="0x011", version = 'V3',
                                   tick_spacing = tick_spacing,
                                   fee = fee)

factory = UniswapFactory("SYS pool factory", "0x2")
lp_low_fee = factory.deploy(exchg_data)

lp_low_fee.precision = 'GWEI'
user_nm = 'user0'

init_sqrt_price_96 = int(1354707084081889255546596745)

lp_low_fee.initialize(init_sqrt_price_96)
print(lp_low_fee.get_price(dai))

None


In [16]:
synthetic_prices_high = np.zeros(len(high))
liquidity_values_high = np.zeros(len(high))

synthetic_prices_low = np.zeros(len(low))
liquidity_values_low = np.zeros(len(low))


In [17]:
n_rows = len(high)

high_fee_lps = high.lp_tuple.unique()
inventory_dict_high = {lp: np.zeros((n_rows, 2)) for lp in high_fee_lps}

fee_dict_high = {lp: np.zeros(n_rows) for lp in high_fee_lps}
in_range_dict = {lp: np.zeros(n_rows) for lp in high_fee_lps}



In [ ]:
for i, row in high.iterrows():
    match row.event:
        case 'mint':
            lp_medium_fee.mint(
                row.liquidity_provider,
                int(row.tickLower),
                int(row.tickUpper),
                int(row.amount_lp_token)
            )
        case 'burn':
           lp_medium_fee.burn(
                row.liquidity_provider,
                int(row.tickLower),
                int(row.tickUpper),
                abs(int(row.amount_lp_token))
            )
        case 'swap':
            if int(row.amount0) > 0:
                token_in = weth
                amount_in = int(row.amount0)
            else:
                token_in = dai
                amount_in = int(row.amount1)
            Swap().apply(lp_medium_fee, token_in, row.trader, amount_in)
            
    synthetic_prices_high[i - 1] = lp_medium_fee.get_price(dai)
    liquidity_values_high[i-1] = lp_medium_fee.get_liquidity_at_tick()
    
    # iterate through all lps and get inventory
    for lp in high_fee_lps:
        owner = lp[0]
        lwr = lp[1]
        upr = lp[2]

        # inventory = lp_medium_fee.get_lp_inventory(owner, lwr, upr)
        # inventory_dict_high[lp][i] = inventory

high['synthetic_price'] = synthetic_prices_high
high['synthetic_liquidity'] = liquidity_values_high


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [118]:
lp_cumsum = high.pivot_table(
    index=high.index, #og index
    columns='lp_tuple', #get all values of this col, unique vals
    values='amount_lp_token', #populate with an agg on this col
    aggfunc='sum' # name the function
).fillna(0) 

cumulative_lp_amounts = lp_cumsum.cumsum() # why did this work? 
col = np.random.choice(lp_cums.columns)
cumulative_lp_amounts

lp_tuple,"(0x, nan, nan)","(0x0035fc5208ef989c28d47e552e92b0c507d2b318, -92100.0, -69060.0)","(0x0111dcc6fc2df10b1145fdcfed2bcf1b1c00de91, 0.0, 1080.0)","(0x018d9b4964a8e428122d2379f04629cc03f35aab, -81600.0, -81480.0)","(0x01b457774fee6105593031a3eb023a873c450c4a, -81840.0, -81240.0)","(0x01bcd8924f864cb4eec6ba6dcf84c35e092ac3f0, -103080.0, -82920.0)","(0x0202ed9ff0d505f9b064a610199a001cef9977bd, -85860.0, -78180.0)","(0x027011c0fd7e86ab7fcab831b204fddfb6af2ed7, -85200.0, -77940.0)","(0x02a1bb37b54d1ce45af90513454e39ff0be70ed8, -81480.0, -81360.0)","(0x03dca0e82c9fd8876245ee54997dd4af73849fd2, -84120.0, -75480.0)",...,"(0xfd7a623fae148e138fb368ed44586dff882199ca, -81360.0, -81240.0)","(0xfd7b50da848f3e3263cfe7f5a2de23530c2cd9fb, -81600.0, -81420.0)","(0xfdc34f61ccc6e5cbf8d168614ab29bae624085c9, -82020.0, -81600.0)","(0xfdd3636caeb29b76fcb9472a93b73d36c7c3f5bf, -82560.0, -79980.0)","(0xfe62145b68ae16b59f88c121651a9019f13f3653, -82380.0, -81540.0)","(0xfe62145b68ae16b59f88c121651a9019f13f3653, -82320.0, -80520.0)","(0xfe62145b68ae16b59f88c121651a9019f13f3653, -81780.0, -81180.0)","(0xff354c7aee506fb65c43a98f57ae65e6b94dd951, -84480.0, -79980.0)","(0xff5269e8ae6a9f2ff2a93c770fd35f8ce71712f8, -82260.0, -81420.0)","(0xffe7e8677cbbbaadd0225be2e12499004681297b, -81480.0, -81300.0)"
0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0
1,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0
2,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0
3,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0
4,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,5.004824e+19,1.901633e+18,9.792049e+18,1.036072e+19,9.947141e+16,0.0,9.463638e+19,0.0,9.449793e+19,...,7.670861e+20,1.800048e+20,1.776022e+19,0.0,0.0,0.0,-2097152.0,9.562045e+19,2.839795e+19,0.0
9996,0.0,5.004824e+19,1.901633e+18,9.792049e+18,1.036072e+19,9.947141e+16,0.0,9.463638e+19,0.0,9.449793e+19,...,7.670861e+20,1.800048e+20,1.776022e+19,0.0,0.0,0.0,-2097152.0,9.562045e+19,2.839795e+19,0.0
9997,0.0,5.004824e+19,1.901633e+18,9.792049e+18,1.036072e+19,9.947141e+16,0.0,9.463638e+19,0.0,9.449793e+19,...,7.670861e+20,1.800048e+20,1.776022e+19,0.0,0.0,0.0,-2097152.0,9.562045e+19,2.839795e+19,0.0
9998,0.0,5.004824e+19,1.901633e+18,9.792049e+18,1.036072e+19,9.947141e+16,0.0,9.463638e+19,0.0,9.449793e+19,...,7.670861e+20,1.800048e+20,1.776022e+19,0.0,0.0,0.0,-2097152.0,9.562045e+19,2.839795e+19,0.0


In [20]:
for i, row in low.iterrows():
    match row.event:
        case 'mint':
            lp_low_fee.mint(
                row.liquidity_provider,
                int(row.tickLower),
                int(row.tickUpper),
                int(row.amount_lp_token)
            )
        case 'burn':
           lp_low_fee.burn(
                row.liquidity_provider,
                int(row.tickLower),
                int(row.tickUpper),
                abs(int(row.amount_lp_token))
            )
        case 'swap':
            if int(row.amount0) > 0:
                token_in = weth
                amount_in = int(row.amount0)
            else:
                token_in = dai
                amount_in = int(row.amount1)
            Swap().apply(lp_low_fee, token_in, row.trader, amount_in)
            
    synthetic_prices_low[i - 1] = lp_low_fee.get_price(dai)
    liquidity_values_low[i-1] = lp_low_fee.get_liquidity_at_tick()
    
low['synthetic_price'] = synthetic_prices_low
low['synthetic_liquidity'] = liquidity_values_low


In [21]:

high['evt_block_time'] = pd.to_datetime(high['evt_block_time'])
low['evt_block_time'] = pd.to_datetime(low['evt_block_time'])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=high['evt_block_time'],
    y=high['synthetic_price'],
    mode='lines',
    name='High Fee Pool Synthetic Price'
))
fig.add_trace(go.Scatter(
    x=high['evt_block_time'],
    y=high['dollar_price'],
    mode='lines',
    name='High Fee Pool Actual Price'
))
fig.add_trace(go.Scatter(
    x=low['evt_block_time'],
    y=low['synthetic_price'],
    mode='lines',
    name='Low Fee Pool Synthetic Price'
))
fig.add_trace(go.Scatter(
    x=low['evt_block_time'],
    y=low['dollar_price'],
    mode='lines',
    name='Low Fee Pool Actual Price'
))

fig.update_layout(
    title="Synthetic vs Actual Dollar Prices for High and Low Fee Pools",
    xaxis_title="Block Time",
    yaxis_title="Price",
    template="plotly_white"
)

fig.show()
